# Identify Flares/Stars with coinciding SPT and TESS observations
This notebook searches TESS for the flaring stars in the SPT flare catalog (Tandoi et.al. 24) and identifies sectors of coinciding observation.
The identifying data of the stars with coinciding observations are saved to `spt_tess_candidates.pkl`.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline
import pandas as pd

from astropy.time import Time
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.io import fits

from scipy.linalg import lstsq
from scipy.optimize import curve_fit

import lightkurve as lk
import tessreduce as tr

import pickle

c:\Users\vishn\miniconda3\envs\SPT_Flares\lib\site-packages\lightkurve\config\__init__.py:119: UserWarning: The default Lightkurve cache directory, used by download(), etc., has been moved to C:\Users\vishn\.lightkurve\cache. Please move all the files in the legacy directory C:\Users\vishn\.lightkurve-cache to the new location and remove the legacy directory. Refer to https://docs.lightkurve.org/reference/config.html#default-cache-directory-migration for more information.
  warnings.warn(


<span style="color:red; font-weight:bold;"> </span>
<span style="color:red; font-weight:bold;"> See these tools for relevant tess sector identification:</span> 
- TESS-point: https://github.com/tessgi/tess-point
  - This would greatly restrict sectors we search for each star, so worth exploring for very large catalogs
- TESS-point Web Tool: https://heasarc.gsfc.nasa.gov/wsgi-scripts/TESS/TESS-point_Web_Tool/TESS-point_Web_Tool/wtv_v2.0.py/

First we retrieve the flare locations and times for SPT and TESS orbit details

The SPT "Flare Star Catalog" is retrieved from https://pole.uchicago.edu/public/data/tandoi24/#Overview.

TESS sector orbit times are retrieved from https://tess.mit.edu/observations/.

In [3]:
spt_flares_df = pd.read_csv('..\data\SPT\spt3g_2019_2022_flare_star_catalog.csv')
spt_flares_df.sort_values(by= 'mjd')
spt_flares_df['sectors'] = [[] for _ in range(len(spt_flares_df))]
print(spt_flares_df.columns)
# print(spt_flares_df[['ra', 'dec']].head())


TESS_sectors_df = pd.read_csv("..\data\TESS\TESS_orbit_times.csv")
# Times are stored by orbit, so first merge by sector
TESS_sectors_df = TESS_sectors_df.groupby("Sector").agg({
    "Start of Orbit": "min",
    "End of Orbit": "max"
}).reset_index()
TESS_sectors_df.columns = ["Sector", "Sector Start", "Sector End"]

# Formatting to MJD for comparison
TESS_sectors_df = TESS_sectors_df.dropna(subset=['Sector Start'])
TESS_sectors_df = TESS_sectors_df.dropna(subset=['Sector Start'])
TESS_sectors_df['Sector'] = TESS_sectors_df['Sector'].apply(lambda x: int(x))
TESS_sectors_df['Sector Start'] = TESS_sectors_df['Sector Start'].apply(lambda x: Time(x, format= 'iso').utc.mjd)
TESS_sectors_df['Sector End'] = TESS_sectors_df['Sector End'].apply(lambda x: Time(x, format= 'iso').utc.mjd)
print(TESS_sectors_df.columns)

Index(['spt_id', 'mjd', 'ts', 'ra', 'dec', '95_flux', '95_flux_err',
       '150_flux', '150_flux_err', '220_flux', '220_flux_err',
       'spectral_index_95_150', 'spectral_index_95_150_err',
       'spectral_index_150_220', 'spectral_index_150_220_err', 'dr3_source_id',
       'p-value', 'source_ra', 'source_dec', 'phot_g_mean_mag',
       'phot_bp_mean_mag', 'phot_rp_mean_mag', 'parallax', 'parallax_err',
       'binary', '2rxs_id', '2rxs_ref', '2rxs_flux', 'sectors'],
      dtype='object')
Index(['Sector', 'Sector Start', 'Sector End'], dtype='object')


Using this data we find the range of TESS sectors over which SPT observed the flares. This is not precise and simply cuts out sectors before the first flare and after the last flare in the catalog.

In [4]:
spt_t_bounds = (spt_flares_df.iloc[0]['mjd'], spt_flares_df.iloc[-1]['mjd'])
t_bounds = Time(spt_t_bounds, format= 'mjd')
print(t_bounds.iso)

# Look for TESS Sectors between first flare start and last flare end
valid_sectors = TESS_sectors_df.loc[(TESS_sectors_df['Sector Start'] <= t_bounds[1].value) & (TESS_sectors_df['Sector End'] >= t_bounds[0].value)]
valid_sectors = set(valid_sectors['Sector'].apply(lambda x: int(x)))
print(valid_sectors)

['2019-04-03 00:28:48.000' '2022-11-04 10:19:12.000']
{10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58}


For each flare in spt_flare_catalog first grab FFIs for all sectors that contain its skycoordinates using tesscut. Lightkurve search is generally fast and I suspect may require less local runtime than checking if flare falls withing explicit time range of each sector.

Then, for the sectors where TESS is looking at the star and check the sector times to identify the one with the flare.

In [5]:
for i, flare in spt_flares_df.iterrows():
    ra = flare['ra']
    dec = flare['dec']
    flare_obs_start = Time(flare['mjd'],  format='mjd', scale='utc')
    c = SkyCoord(ra, dec, unit= 'deg')

    # Search for any sectors where TESS is looking at the star
    possible_matches = lk.search_tesscut(c, [sector for sector in valid_sectors])
    if len(possible_matches) >= 1:
        # Check time range for each sector
        for data_prod in possible_matches:
            sector = int(data_prod.mission[0][11:])
            sector_time = TESS_sectors_df[TESS_sectors_df['Sector'] == sector]
            sector_start = sector_time['Sector Start'].iloc[0]
            sector_end = sector_time['Sector End'].iloc[0]
            if sector_start < flare_obs_start.value < sector_end:
                print(f'{i}/{spt_flares_df.shape[0]}: Match found for Sector {sector}')
                flare['sectors'] = flare['sectors'].append(sector)
            else:
                print(f'{i}/{spt_flares_df.shape[0]}: Sector {sector} - no overlap in time')
    else:
        print(f'{i}/{spt_flares_df.shape[0]}: no nearby targets')


0/111: Sector 28 - no overlap in time
0/111: Sector 29 - no overlap in time
1/111: Sector 30 - no overlap in time
1/111: Sector 28 - no overlap in time
1/111: Sector 29 - no overlap in time
2/111: Sector 30 - no overlap in time
2/111: Sector 29 - no overlap in time
3/111: Sector 29 - no overlap in time
4/111: Sector 27 - no overlap in time
5/111: Sector 29 - no overlap in time
6/111: Sector 29 - no overlap in time
7/111: Sector 28 - no overlap in time
8/111: Sector 27 - no overlap in time
9/111: Sector 27 - no overlap in time
10/111: Sector 27 - no overlap in time
11/111: Sector 30 - no overlap in time
11/111: Sector 29 - no overlap in time
12/111: Sector 28 - no overlap in time
13/111: Sector 28 - no overlap in time
13/111: Sector 29 - no overlap in time
14/111: Sector 30 - no overlap in time
14/111: Sector 29 - no overlap in time
15/111: Sector 28 - no overlap in time
15/111: Sector 29 - no overlap in time
16/111: Sector 30 - no overlap in time
16/111: Sector 29 - no overlap in time


In [6]:
spt_tess_candidates = spt_flares_df[spt_flares_df['sectors'].apply(lambda x: len(x) > 0)].reset_index()
spt_tess_candidates

,index,spt_id,mjd,ts,ra,dec,95_flux,95_flux_err,150_flux,150_flux_err,...,phot_g_mean_mag,phot_bp_mean_mag,phot_rp_mean_mag,parallax,parallax_err,binary,2rxs_id,2rxs_ref,2rxs_flux,sectors
0,42,SPT-SV J232857.8-680230,59067.63,69.829680,352.241137,-68.041907,33.132387,6.050987,44.396645,6.479653,...,11.975413,13.298294,10.835975,21.814973,0.016096,True,2RXS J232857.7-680225,Freund,0.0,[28]
1,43,SPT-SV J220610.2-460354,59068.28,65.720511,331.542835,-46.065046,47.818806,8.045753,50.842904,8.377214,...,11.180600,11.394999,10.820612,2.686802,0.032307,False,NaN,NaN,NaN,[28]
2,46,SPT-SV J025532.4-570257,59110.27,1233.665708,43.885140,-57.049441,162.083004,7.148120,206.963060,7.665012,...,14.370894,16.325182,13.060138,21.889704,0.021013,False,NaN,NaN,NaN,[29]
3,49,SPT-SV J014432.2-460435,59131.78,48.975169,26.134435,-46.076393,45.598152,8.294850,36.632626,8.695313,...,15.635771,18.198709,14.208293,25.949235,0.045132,False,NaN,NaN,NaN,[30]


Print out flare skycoord in hmsdms to search in SINBAD and find common identifier for flares; I chose TIC.

In [7]:
spt_tess_candidates[['spt_id', 'dr3_source_id']]

for i, flare in spt_tess_candidates.iterrows():
    ra = flare['ra']
    dec = flare['dec']
    c = SkyCoord(ra, dec, frame= 'icrs', unit= 'deg')
    print(i, c.to_string('hmsdms'))

0 23h28m57.87288s -68d02m30.8652s
1 22h06m10.2804s -46d03m54.1656s
2 02h55m32.4336s -57d02m57.9876s
3 01h44m32.2644s -46d04m35.0148s


Add TIC to dataframe

In [8]:
spt_tess_candidates['TIC'] = spt_tess_candidates['spt_id']

spt_tess_candidates.at[0,'TIC'] = "TIC229807000"
spt_tess_candidates.at[1,'TIC'] = "TIC279255411" #TIC279255412
spt_tess_candidates.at[2,'TIC'] = "TIC207138379"
spt_tess_candidates.at[3,'TIC'] = "TIC401838575" 


spt_tess_candidates[['spt_id', 'dr3_source_id', 'TIC', 'sectors']]

,spt_id,dr3_source_id,TIC,sectors
0,SPT-SV J232857.8-680230,6388014157668558080,TIC229807000,[28]
1,SPT-SV J220610.2-460354,6567445491726233216,TIC279255411,[28]
2,SPT-SV J025532.4-570257,4727920241619206912,TIC207138379,[29]
3,SPT-SV J014432.2-460435,4954453580066220800,TIC401838575,[30]


Save dataframe

In [ ]:
spt_tess_candidates.to_pickle('..\data\spt_tess_candidates_tandoi24.pkl')